In [1]:
import numpy as np
from irstdet.utils import GenericMask
from irstdet.utils.box_mode import BoxMode
import os.path as osp
import mmengine.fileio as fileio
from sahi.utils.file import save_json
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation,CocoPrediction
import os
from PIL import Image
import json
import numpy as np
from pycocotools import mask as pymask
from skimage import measure
from tqdm import tqdm


In [2]:
data_root = "./data/"
data_names= ['IRSTD-1K', 'NUDT-SIRST', 'NUAA-SIRST']
imgs_dir = 'images'
labels_dir =  'masks'
ind = 'img_idx'


In [9]:
train_coco = Coco(image_id_setting='manual')
train_coco.add_category(CocoCategory(id=1, name='target'))

test_coco = Coco(image_id_setting='manual')
test_coco.add_category(CocoCategory(id=1, name='target'))

id = 0

data_name_bar = tqdm(data_names)
for data_name in data_name_bar:
    data_name_bar.set_description(f"Processing {data_name}")
    current_data_root = osp.join(data_root, data_name) # ./data/IRSTD-1K
    # img_names = [i for i in fileio.list_from_file(osp.join(data_root, data_name, ))]
    imgs_list = [i for i in fileio.list_dir_or_file(osp.join(current_data_root, imgs_dir))] # ['xxx.png']
    masks_list = [i for i in fileio.list_dir_or_file(osp.join(current_data_root, labels_dir))] # # ['xxx.png']
    
    train_ind = [i for i in fileio.list_from_file(osp.join(current_data_root, ind, f'train_{data_name}.txt' ))]
    test_ind = [i for i in fileio.list_from_file(osp.join(current_data_root, ind, f'test_{data_name}.txt' ))]

    img_name_bar = tqdm(imgs_list)
    for img_file in img_name_bar:
        img_name_bar.set_description(f"Processing {img_file}")
        img_path = osp.join(current_data_root, imgs_dir, img_file)
        mask_path = osp.join(current_data_root, labels_dir, img_file)
        mask = Image.open(mask_path).convert('L')
        w, h= mask.size

        coco_image = CocoImage(
        file_name=f"{osp.join(data_name, imgs_dir, img_file)}", height=h, width=w, id=int(id))


        fotran_mask = np.asfortranarray(np.array(mask))
        encoded_ground_truth = pymask.encode(np.array(fotran_mask))
        contours = measure.find_contours(np.array(fotran_mask), 0.5)
        
        segmentations = []

        for contour in contours:
            contour = np.flip(contour, axis=1)
            segmentation = contour.ravel().tolist()
            coco_image.add_annotation(
            CocoAnnotation(
                category_id=1,
                category_name='target',
                segmentation=[segmentation],
                image_id=int(id)
            )
        )
        try :
            if img_file.split('.')[0] in train_ind:
                train_coco.add_image(coco_image)
            elif img_file.split('.')[0] in test_ind: 
                test_coco.add_image(coco_image)
            else: 
                raise ValueError(f"Image {img_file} not found in either images or masks list")
        except:
            print(f"Image {img_file} not found in either images or masks list")
        id+=1
        


Processing IRSTD-1K:   0%|          | 0/3 [00:00<?, ?it/s]

Processing NUAA-SIRST: 100%|██████████| 3/3 [00:28<00:00,  9.38s/it]


In [10]:
fileio.dump(train_coco.json,osp.join(data_root, 'train.json'))
fileio.dump(test_coco.json, osp.join(data_root, 'test.json'))

In [6]:
img_file in imgs_list

True

In [7]:
train_coco.stats

{'num_images': 1676,
 'num_annotations': 2483,
 'num_categories': 1,
 'num_negative_images': 0,
 'num_images_per_category': {'target': 1676},
 'num_annotations_per_category': {'target': 2483},
 'min_num_annotations_in_image': 1,
 'max_num_annotations_in_image': 7,
 'avg_num_annotations_in_image': 1.4815035799522673,
 'min_annotation_area': 0,
 'max_annotation_area': 1111,
 'avg_annotation_area': 50.35440998791784,
 'min_annotation_area_per_category': {'target': 0},
 'max_annotation_area_per_category': {'target': 1111}}

In [8]:
test_coco.stats

{'num_images': 1079,
 'num_annotations': 1564,
 'num_categories': 1,
 'num_negative_images': 0,
 'num_images_per_category': {'target': 1079},
 'num_annotations_per_category': {'target': 1564},
 'min_num_annotations_in_image': 1,
 'max_num_annotations_in_image': 8,
 'avg_num_annotations_in_image': 1.4494902687673772,
 'min_annotation_area': 0,
 'max_annotation_area': 1183,
 'avg_annotation_area': 42.577365728900254,
 'min_annotation_area_per_category': {'target': 0},
 'max_annotation_area_per_category': {'target': 1183}}